# Package installation

In [2]:
!pip install pandas
!pip install scikit-learn
!pip install --upgrade tensorflow keras numpy pandas sklearn pillow

     |████████████████████████████████| 9.5MB 16.5MB/s eta 0:00:01
     |████████████████████████████████| 512kB 54.1MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 6.8MB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 25.9MB 52.0MB/s eta 0:00:01
     |████████████████████████████████| 307kB 64.0MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 320.4MB 163kB/s  eta 0:00:01     |█████████████████████████████▌  | 295.3MB 52.4MB/s eta 0:00:01
     |████████████████████████████████| 14.5MB 40.7MB/s eta 0:00:01
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.4)
     |████████████████████████████████| 2.2MB 35.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 32.5MB/s eta 0:00:01
     |████████████████████████████████| 10.6MB 20.3MB/s eta 0:00:01
     |█████████████

In [12]:
!pip install matplotlib

You should consider upgrading via the 'pip install --upgrade pip' command.


### import packages

In [3]:
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K
from keras.applications.resnet_v2 import ResNet50V2

from datetime import datetime
import time
import os
import numpy as np
import tensorflow as tf
import re
import pandas as pd

from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers

from tensorflow.keras.optimizers import RMSprop

from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# 1st round of training: Freeze all layers

## Setting up model

In [5]:
learning_rate = 1e-4
img_height, img_width = 224, 224
filename = "resnet_freeze_all"

In [6]:
pre_trained_model = ResNet50V2( input_shape=(img_height,img_width,3),
                                include_top = False,
                                weights = 'imagenet' 
)

94674944/94668760 [==============================] - 2s 0us/step


In [7]:
#Make all layers non-trainable
for layer in pre_trained_model.layers:
    layer.trainable= False

In [8]:
# Flatten the output layer to 1-dimension
x = layers.Flatten()(pre_trained_model.output)
# Add fully connected layer, with relu activation
x = layers.Dense(1024, activation='relu')(x)
# Add dropout
x = layers.Dropout(0.5)(x)
# Add sigmoid layer for classification. Sigmoid is used instead of softmax, because this is a binary classifier
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

# binary crossentropy used for binary classification
model.compile(optimizer=RMSprop(learning_rate),
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

## Train generator

In [10]:
batch_size = 256
train_directory = "storage/train"
validation_directory = "storage/validate"
seed = 10
no_epochs = 40

# Data augmentation
train_datagen =ImageDataGenerator(rescale=1./255,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1)
train_generator = train_datagen.flow_from_directory(
    directory=train_directory, 
    batch_size = batch_size,
    class_mode ='binary',
    target_size = (img_height,img_width),
shuffle = True)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_directory(
    directory=validation_directory, 
    class_mode ='binary',
    target_size = (img_height,img_width))

Found 8500 images belonging to 2 classes.
Found 1061 images belonging to 2 classes.


In [13]:
checkpoint_filepath = f"storage/checkpoint/{filename}.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit_generator(generator=train_generator,
                    validation_data=validation_generator, 
                    epochs=no_epochs, 
                    verbose=1,
                   callbacks=[model_checkpoint_callback]
                   )

Epoch 1/40
34/34 [==============================] - 298s 9s/step - loss: 0.6710 - accuracy: 0.6758 - val_loss: 0.5184 - val_accuracy: 0.7370
Epoch 2/40
34/34 [==============================] - 299s 9s/step - loss: 0.6086 - accuracy: 0.7131 - val_loss: 0.5046 - val_accuracy: 0.7333
Epoch 3/40
34/34 [==============================] - 297s 9s/step - loss: 0.5664 - accuracy: 0.7251 - val_loss: 1.0524 - val_accuracy: 0.6032
Epoch 4/40
34/34 [==============================] - 302s 9s/step - loss: 0.5016 - accuracy: 0.7678 - val_loss: 0.4883 - val_accuracy: 0.7653
Epoch 5/40
34/34 [==============================] - 296s 9s/step - loss: 0.4813 - accuracy: 0.7759 - val_loss: 0.5640 - val_accuracy: 0.7427
Epoch 6/40
34/34 [==============================] - 300s 9s/step - loss: 0.4794 - accuracy: 0.7827 - val_loss: 0.3943 - val_accuracy: 0.8143
Epoch 7/40
34/34 [==============================] - 299s 9s/step - loss: 0.4740 - accuracy: 0.7920 - val_loss: 0.4438 - val_accuracy: 0.7983
Epoch 8/40
34

In [15]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss= history.history['loss']
val_loss = history.history['val_loss']

plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(f'storage/results/{filename}_accuracy.png')
plt.close()

plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(f'storage/results/{filename}_loss.png')
plt.close()

## Get confusion matrix

In [31]:
#load best model
model.load_weights(f"storage/checkpoint/{filename}.hdf5")

In [32]:
from sklearn.metrics import confusion_matrix

test_directory = "storage/test/"

test_datagen = ImageDataGenerator(rescale=1./255)

generator = test_datagen.flow_from_directory(
        test_directory,
        target_size=(224, 224),
        batch_size=16,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

probabilities = model.predict_generator(generator, verbose =1)

Found 1635 images belonging to 2 classes.
103/103 [==============================] - 45s 433ms/step


In [33]:
import os

num_female = sum(len(files) for _, _, files in os.walk('storage/test/female'))
num_male = sum(len(files) for _, _, files in os.walk('storage/test/male'))
print(num_female + num_male)

1635


In [34]:
y_true = np.array([0] * num_female + [1] * num_male)
y_pred = probabilities > 0.5

res = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots()
im = ax.imshow(res, cmap='Blues')
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, res[i, j],
                       ha="center", va="center", color="y")
plt.title('Confusion matrix from model prediction')
plt.savefig(f'storage/results/{filename}_confusion_matrix.png')
plt.close()

print(res)

[[785 101]
 [105 644]]


# 2nd round of training: Unfreeze last block

## Setting up model

In [ ]:
learning_rate = 1e-4
img_height, img_width = 224, 224
filename = "resnet_fine_tune"

In [ ]:
pre_trained_model = ResNet50V2( input_shape=(img_height,img_width,3),
                                include_top = False,
                                weights = 'imagenet'        
)

In [ ]:
#Unfreeze last 38 layers
for layer in pre_trained_model.layers[:-38]:
    layer.trainable= False
for layer in pre_trained_model.layers[-38:]:
    layer.trainable= True

In [ ]:
# Flatten the output layer to 1-dimension
x = layers.Flatten()(pre_trained_model.output)
# Add fully connected layer, with relu activation
x = layers.Dense(1024, activation='relu')(x)
# Add dropout
x = layers.Dropout(0.5)(x)
# Add sigmoid layer for classification. Sigmoid is used instead of softmax, because this is a binary classifier
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

# binary crossentropy used for binary classification
model.compile(optimizer=RMSprop(learning_rate),
             loss = 'binary_crossentropy',
             metrics=['accuracy'])

## Setting up train and validation generators

In [10]:
batch_size = 256
train_directory = "storage/train"
validation_directory = "storage/validate"
seed = 10
no_epochs = 40

# Data augmentation
train_datagen =ImageDataGenerator(rescale=1./255,
                                 width_shift_range = 0.1,
                                 height_shift_range = 0.1)
train_generator = train_datagen.flow_from_directory(
    directory=train_directory, 
    batch_size = batch_size,
    class_mode ='binary',
    target_size = (img_height,img_width),
shuffle = True)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_directory(
    directory=validation_directory, 
    class_mode ='binary',
    target_size = (img_height,img_width))

Found 8500 images belonging to 2 classes.
Found 1061 images belonging to 2 classes.


In [11]:
checkpoint_filepath = f"storage/checkpoint/{filename}.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit_generator(generator=train_generator,
                    validation_data=validation_generator, 
                    epochs=no_epochs, 
                    verbose=1,
                   callbacks=[model_checkpoint_callback]
                   )

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
34/34 [==============================] - 486s 14s/step - loss: 1.5426 - accuracy: 0.7412 - val_loss: 0.7147 - val_accuracy: 0.7418
Epoch 2/40
34/34 [==============================] - 479s 14s/step - loss: 0.3422 - accuracy: 0.8505 - val_loss: 0.3803 - val_accuracy: 0.8699
Epoch 3/40
34/34 [==============================] - 482s 14s/step - loss: 0.2083 - accuracy: 0.9139 - val_loss: 0.4254 - val_accuracy: 0.8954
Epoch 4/40
34/34 [==============================] - 480s 14s/step - loss: 0.1555 - accuracy: 0.9369 - val_loss: 0.5016 - val_accuracy: 0.9029
Epoch 5/40
34/34 [==============================] - 482s 14s/step - loss: 0.1013 - accuracy: 0.9605 - val_loss: 0.5575 - val_accuracy: 0.9171
Epoch 6/40
34/34 [==============================] - 478s 14s/step - loss: 0.0787 - accuracy: 0.9721 - val_loss: 0.8274 - val_accuracy: 0.8869
Epoch 7/40
34/34 [==============================] - 481s 14s/step - loss

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss= history.history['loss']
val_loss = history.history['val_loss']

plt.plot(accuracy)
plt.plot(val_accuracy)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(f'storage/results/{filename}_accuracy.png')
plt.close()

plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig(f'storage/results/{filename}_loss.png')
plt.close()

## Get confusion matrix

In [ ]:
#load best model
model.load_weights(f"storage/checkpoint/{filename}.hdf5")

In [14]:
from sklearn.metrics import confusion_matrix

test_directory = "storage/test/"

test_datagen = ImageDataGenerator(rescale=1./255)

generator = test_datagen.flow_from_directory(
        test_directory,
        target_size=(224, 224),
        batch_size=16,
        class_mode=None,  # only data, no labels
        shuffle=False)  # keep data in same order as labels

probabilities = model.predict_generator(generator, verbose =1)

Found 1635 images belonging to 2 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
103/103 [==============================] - 40s 388ms/step


In [15]:
import os

num_female = sum(len(files) for _, _, files in os.walk('storage/test/female'))
num_male = sum(len(files) for _, _, files in os.walk('storage/test/male'))
print(num_female + num_male)

1635


In [16]:
y_true = np.array([0] * num_female + [1] * num_male)
y_pred = probabilities > 0.5

res = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots()
im = ax.imshow(res, cmap='Blues')
for i in range(2):
    for j in range(2):
        text = ax.text(j, i, res[i, j],
                       ha="center", va="center", color="y")
plt.title('Confusion matrix from model prediction')
plt.savefig(f'storage/results/{filename}_confusion_matrix.png')
plt.close()

print(res)

[[835  51]
 [ 73 676]]
